# Interpolasi

In [1]:
import pandas as pd

df_varietas = pd.read_excel('./dataset/split_data/varietas_topografi.xlsx')

df_lahan = pd.read_excel('./dataset/split_data/lahan_topografi.xlsx')

def interpolasi_ketinggian(x, min_val,max_val):
    if  min_val <= x <= max_val:
        return 5
    elif 0 <= x < min_val:
        return 1 + ((x - 0) / (min_val - 0)) * (5 - 1)
    elif max_val < x <= (min_val + max_val):
        return 5 + ((x - max_val) / ((min_val + max_val) - max_val)) * (1 - 5)
    else:
        return 1

def interpolasi_kemiringan(x,min_val, max_val):
    if min_val <= x <= max_val:
        return 5
    elif max_val < x <= 80 :
        return 5 + ((x - max_val) / (80  - max_val)) * (1 - 5)
    else: 
        return 1

def interpolasi_drainase(x, value):
    if x == value:
        return 5
    else:
        return 1
    
def hitung_topografi(row, jenis_varietas):
    var_data = df_varietas[df_varietas['jenis_varietas'] == jenis_varietas].iloc[0]

    hasil = {
        'ketinggian': interpolasi_ketinggian(row['ketinggian'], var_data['ketinggian_min'], var_data['ketinggian_max']),
        'kemiringan' : interpolasi_kemiringan(row['kemiringan'], var_data['kemiringan_min'], var_data['kemiringan_max']),
        'drainase': interpolasi_drainase(row['drainase'], var_data['drainase'])
    }

    return hasil

def skor_V1(row):
    return hitung_topografi(row, 'V1')

def skor_V2(row):
    return hitung_topografi(row, 'V2')

def skor_V3(row):
    return hitung_topografi(row, 'V3')

def skor_V4(row):
    return hitung_topografi(row, 'V4')

def skor_V5(row):
    return hitung_topografi(row, 'V5')


# Bobot Utama

In [2]:
bobot_utama = pd.read_excel('./dataset/bobot_ahp/parameter_utama.xlsx')
bobot_utama_dict = bobot_utama.set_index('Label')['Weight'].to_dict()
bobot_utama_dict

{'iklim': 0.4053875480953168,
 'topografi': 0.1139723667233914,
 'tanah': 0.4806400851812919}

## Bobot AHP Parameter Topografi

In [3]:
bobot_topo = pd.read_excel('./dataset/bobot_ahp/parameter_topografi.xlsx')
bobot_topo_dict = bobot_topo.set_index('Label')['Weight'].to_dict()
bobot_topo_dict

{'ketinggian': 0.4806400851812919,
 'kemiringan': 0.1139723667233914,
 'drainase': 0.4053875480953168}

# Varietas 1

In [4]:
lahan_scores = df_lahan.apply(skor_V1, axis=1)

topografi_V1 = pd.concat([df_lahan['nama_lahan'], lahan_scores.apply(pd.Series)], axis=1)

topografi_V1

,nama_lahan,ketinggian,kemiringan,drainase
0,Bakung,2.280,5.000000,5.0
1,Binangun,2.944,5.000000,5.0
2,Doko,3.792,5.000000,5.0
3,Gandusari,3.784,5.000000,5.0
4,Garum,3.208,5.000000,5.0
5,Kademangan,2.968,5.000000,1.0
6,Kanigoro,2.464,5.000000,5.0
7,Kesamben,2.776,5.000000,5.0
8,Nglegok,3.600,5.000000,5.0
9,Panggungrejo,3.480,3.857143,5.0


### SUM (perkalian V1 dengan bobot AHP)

In [5]:
skor_topo_V1 = topografi_V1.copy()

for kolom in ['ketinggian','kemiringan','drainase']:
    skor_topo_V1[kolom] = skor_topo_V1[kolom] * bobot_topo_dict[kolom]

# Menambahkan kolom 'total_skor' yang berisi jumlah dari kolom yang telah dikalikan bobot
skor_topo_V1['total_skor'] = skor_topo_V1[['ketinggian','kemiringan','drainase']].sum(axis=1)

skor_topo_V1

,nama_lahan,ketinggian,kemiringan,drainase,total_skor
0,Bakung,1.095859,0.569862,2.026938,3.692659
1,Binangun,1.415004,0.569862,2.026938,4.011804
2,Doko,1.822587,0.569862,2.026938,4.419387
3,Gandusari,1.818742,0.569862,2.026938,4.415542
4,Garum,1.541893,0.569862,2.026938,4.138693
5,Kademangan,1.426540,0.569862,0.405388,2.401789
6,Kanigoro,1.184297,0.569862,2.026938,3.781097
7,Kesamben,1.334257,0.569862,2.026938,3.931056
8,Nglegok,1.730304,0.569862,2.026938,4.327104
9,Panggungrejo,1.672627,0.439608,2.026938,4.139173


### Total topografi V1 * Bobot AHP topografi

In [6]:
# Mengalikan kolom 'total_skor' dengan bobot 'iklim'
total_skor_topo_V1 = skor_topo_V1.copy()
total_skor_topo_V1['total_skor_topo_V1'] = total_skor_topo_V1['total_skor'] * bobot_utama_dict['topografi']
total_skor_topo_V1


,nama_lahan,ketinggian,kemiringan,drainase,total_skor,total_skor_topo_V1
0,Bakung,1.095859,0.569862,2.026938,3.692659,0.420861
1,Binangun,1.415004,0.569862,2.026938,4.011804,0.457235
2,Doko,1.822587,0.569862,2.026938,4.419387,0.503688
3,Gandusari,1.818742,0.569862,2.026938,4.415542,0.503250
4,Garum,1.541893,0.569862,2.026938,4.138693,0.471697
5,Kademangan,1.426540,0.569862,0.405388,2.401789,0.273738
6,Kanigoro,1.184297,0.569862,2.026938,3.781097,0.430941
7,Kesamben,1.334257,0.569862,2.026938,3.931056,0.448032
8,Nglegok,1.730304,0.569862,2.026938,4.327104,0.493170
9,Panggungrejo,1.672627,0.439608,2.026938,4.139173,0.471751


# Varietas V2

In [7]:
lahan_scores = df_lahan.apply(skor_V2, axis=1)

topografi_V2 = pd.concat([df_lahan['nama_lahan'], lahan_scores.apply(pd.Series)], axis=1)

topografi_V2

,nama_lahan,ketinggian,kemiringan,drainase
0,Bakung,5.00,5.000000,5.0
1,Binangun,5.00,5.000000,5.0
2,Doko,3.04,5.000000,5.0
3,Gandusari,3.08,5.000000,5.0
4,Garum,5.00,5.000000,5.0
5,Kademangan,5.00,5.000000,1.0
6,Kanigoro,5.00,5.000000,5.0
7,Kesamben,5.00,5.000000,5.0
8,Nglegok,4.00,5.000000,5.0
9,Panggungrejo,4.60,3.857143,5.0


### SUM (perkalian V2 dengan bobot AHP)

In [8]:
skor_topo_V2 = topografi_V2.copy()

for kolom in ['ketinggian','kemiringan','drainase']:
    skor_topo_V2[kolom] = skor_topo_V2[kolom] * bobot_topo_dict[kolom]

skor_topo_V2['total_skor'] = skor_topo_V2[['ketinggian','kemiringan','drainase']].sum(axis=1)

skor_topo_V2

,nama_lahan,ketinggian,kemiringan,drainase,total_skor
0,Bakung,2.403200,0.569862,2.026938,5.000000
1,Binangun,2.403200,0.569862,2.026938,5.000000
2,Doko,1.461146,0.569862,2.026938,4.057945
3,Gandusari,1.480371,0.569862,2.026938,4.077171
4,Garum,2.403200,0.569862,2.026938,5.000000
5,Kademangan,2.403200,0.569862,0.405388,3.378450
6,Kanigoro,2.403200,0.569862,2.026938,5.000000
7,Kesamben,2.403200,0.569862,2.026938,5.000000
8,Nglegok,1.922560,0.569862,2.026938,4.519360
9,Panggungrejo,2.210944,0.439608,2.026938,4.677490


### Total topografi V2 * Bobot AHP topografi

In [19]:
# Mengalikan kolom 'total_skor' dengan bobot 'iklim'
total_skor_topo_V2 = skor_topo_V2.copy()
total_skor_topo_V2['total_skor_topo_V2'] = total_skor_topo_V2['total_skor'] * bobot_utama_dict['topografi']
total_skor_topo_V2


,nama_lahan,ketinggian,kemiringan,drainase,total_skor,total_skor_topo_V2
0,Bakung,2.403200,0.569862,2.026938,5.000000,0.569862
1,Binangun,2.403200,0.569862,2.026938,5.000000,0.569862
2,Doko,1.461146,0.569862,2.026938,4.057945,0.462494
3,Gandusari,1.480371,0.569862,2.026938,4.077171,0.464685
4,Garum,2.403200,0.569862,2.026938,5.000000,0.569862
5,Kademangan,2.403200,0.569862,0.405388,3.378450,0.385050
6,Kanigoro,2.403200,0.569862,2.026938,5.000000,0.569862
7,Kesamben,2.403200,0.569862,2.026938,5.000000,0.569862
8,Nglegok,1.922560,0.569862,2.026938,4.519360,0.515082
9,Panggungrejo,2.210944,0.439608,2.026938,4.677490,0.533105


# Varietas V3

In [9]:
lahan_scores = df_lahan.apply(skor_V3, axis=1)

topografi_V3 = pd.concat([df_lahan['nama_lahan'], lahan_scores.apply(pd.Series)], axis=1)

topografi_V3

,nama_lahan,ketinggian,kemiringan,drainase
0,Bakung,5.0,5.000000,5.0
1,Binangun,5.0,5.000000,5.0
2,Doko,1.0,5.000000,5.0
3,Gandusari,1.0,5.000000,5.0
4,Garum,5.0,5.000000,5.0
5,Kademangan,5.0,5.000000,1.0
6,Kanigoro,5.0,5.000000,5.0
7,Kesamben,5.0,5.000000,5.0
8,Nglegok,1.0,5.000000,5.0
9,Panggungrejo,1.0,3.857143,5.0


### SUM (Perkalian V3 dengan bobot AHP)

In [10]:
skor_topo_V3 = topografi_V3.copy()

for kolom in ['ketinggian','kemiringan','drainase']:
    skor_topo_V3[kolom] = skor_topo_V3[kolom] * bobot_topo_dict[kolom]

skor_topo_V3['total_skor'] = skor_topo_V3[['ketinggian','kemiringan','drainase']].sum(axis=1)

skor_topo_V3

,nama_lahan,ketinggian,kemiringan,drainase,total_skor
0,Bakung,2.40320,0.569862,2.026938,5.000000
1,Binangun,2.40320,0.569862,2.026938,5.000000
2,Doko,0.48064,0.569862,2.026938,3.077440
3,Gandusari,0.48064,0.569862,2.026938,3.077440
4,Garum,2.40320,0.569862,2.026938,5.000000
5,Kademangan,2.40320,0.569862,0.405388,3.378450
6,Kanigoro,2.40320,0.569862,2.026938,5.000000
7,Kesamben,2.40320,0.569862,2.026938,5.000000
8,Nglegok,0.48064,0.569862,2.026938,3.077440
9,Panggungrejo,0.48064,0.439608,2.026938,2.947186


### Total topografi V3 * Bobot AHP topografi

In [11]:
# Mengalikan kolom 'total_skor' dengan bobot 'iklim'
total_skor_topo_V3 = skor_topo_V3.copy()
total_skor_topo_V3['total_skor_topo_V3'] = total_skor_topo_V3['total_skor'] * bobot_utama_dict['topografi']
total_skor_topo_V3


,nama_lahan,ketinggian,kemiringan,drainase,total_skor,total_skor_topo_V3
0,Bakung,2.40320,0.569862,2.026938,5.000000,0.569862
1,Binangun,2.40320,0.569862,2.026938,5.000000,0.569862
2,Doko,0.48064,0.569862,2.026938,3.077440,0.350743
3,Gandusari,0.48064,0.569862,2.026938,3.077440,0.350743
4,Garum,2.40320,0.569862,2.026938,5.000000,0.569862
5,Kademangan,2.40320,0.569862,0.405388,3.378450,0.385050
6,Kanigoro,2.40320,0.569862,2.026938,5.000000,0.569862
7,Kesamben,2.40320,0.569862,2.026938,5.000000,0.569862
8,Nglegok,0.48064,0.569862,2.026938,3.077440,0.350743
9,Panggungrejo,0.48064,0.439608,2.026938,2.947186,0.335898


# Varietas 4

In [13]:
lahan_scores = df_lahan.apply(skor_V4, axis=1)

topografi_V4 = pd.concat([df_lahan['nama_lahan'], lahan_scores.apply(pd.Series)], axis=1)

topografi_V4

,nama_lahan,ketinggian,kemiringan,drainase
0,Bakung,5.0,3.8,5.0
1,Binangun,5.0,3.8,5.0
2,Doko,1.0,5.0,5.0
3,Gandusari,1.0,5.0,5.0
4,Garum,5.0,5.0,5.0
5,Kademangan,5.0,5.0,1.0
6,Kanigoro,5.0,5.0,5.0
7,Kesamben,5.0,5.0,5.0
8,Nglegok,1.0,5.0,5.0
9,Panggungrejo,1.0,3.0,5.0


### SUM (perkalian V4 dengan bobot AHP)

In [14]:
skor_topo_V4 = topografi_V4.copy()

for kolom in ['ketinggian','kemiringan','drainase']:
    skor_topo_V4[kolom] = skor_topo_V4[kolom] * bobot_topo_dict[kolom]

skor_topo_V4['total_skor'] = skor_topo_V4[['ketinggian','kemiringan','drainase']].sum(axis=1)

skor_topo_V4

,nama_lahan,ketinggian,kemiringan,drainase,total_skor
0,Bakung,2.40320,0.433095,2.026938,4.863233
1,Binangun,2.40320,0.433095,2.026938,4.863233
2,Doko,0.48064,0.569862,2.026938,3.077440
3,Gandusari,0.48064,0.569862,2.026938,3.077440
4,Garum,2.40320,0.569862,2.026938,5.000000
5,Kademangan,2.40320,0.569862,0.405388,3.378450
6,Kanigoro,2.40320,0.569862,2.026938,5.000000
7,Kesamben,2.40320,0.569862,2.026938,5.000000
8,Nglegok,0.48064,0.569862,2.026938,3.077440
9,Panggungrejo,0.48064,0.341917,2.026938,2.849495


### Total topografi V4 * Bobot AHP topografi

In [15]:
# Mengalikan kolom 'total_skor' dengan bobot 'iklim'
total_skor_topo_V4 = skor_topo_V4.copy()
total_skor_topo_V4['total_skor_topo_V4'] = total_skor_topo_V4['total_skor'] * bobot_utama_dict['topografi']
total_skor_topo_V4


,nama_lahan,ketinggian,kemiringan,drainase,total_skor,total_skor_topo_V4
0,Bakung,2.40320,0.433095,2.026938,4.863233,0.554274
1,Binangun,2.40320,0.433095,2.026938,4.863233,0.554274
2,Doko,0.48064,0.569862,2.026938,3.077440,0.350743
3,Gandusari,0.48064,0.569862,2.026938,3.077440,0.350743
4,Garum,2.40320,0.569862,2.026938,5.000000,0.569862
5,Kademangan,2.40320,0.569862,0.405388,3.378450,0.385050
6,Kanigoro,2.40320,0.569862,2.026938,5.000000,0.569862
7,Kesamben,2.40320,0.569862,2.026938,5.000000,0.569862
8,Nglegok,0.48064,0.569862,2.026938,3.077440,0.350743
9,Panggungrejo,0.48064,0.341917,2.026938,2.849495,0.324764


# Varietas V5

In [16]:
lahan_scores = df_lahan.apply(skor_V5, axis=1)

topografi_V5 = pd.concat([df_lahan['nama_lahan'], lahan_scores.apply(pd.Series)], axis=1)

topografi_V5

,nama_lahan,ketinggian,kemiringan,drainase
0,Bakung,2.60,5.000000,5.0
1,Binangun,3.43,5.000000,5.0
2,Doko,4.49,5.000000,5.0
3,Gandusari,4.48,5.000000,5.0
4,Garum,3.76,5.000000,5.0
5,Kademangan,3.46,5.000000,1.0
6,Kanigoro,2.83,5.000000,5.0
7,Kesamben,3.22,5.000000,5.0
8,Nglegok,4.25,5.000000,5.0
9,Panggungrejo,4.10,3.857143,5.0


### SUM (perkalian V5 dengan bobot AHP)

In [17]:
skor_topo_V5 = topografi_V5.copy()

for kolom in ['ketinggian','kemiringan','drainase']:
    skor_topo_V5[kolom] = skor_topo_V5[kolom] * bobot_topo_dict[kolom]

skor_topo_V5['total_skor'] = skor_topo_V5[['ketinggian','kemiringan','drainase']].sum(axis=1)

skor_topo_V5

,nama_lahan,ketinggian,kemiringan,drainase,total_skor
0,Bakung,1.249664,0.569862,2.026938,3.846464
1,Binangun,1.648595,0.569862,2.026938,4.245395
2,Doko,2.158074,0.569862,2.026938,4.754874
3,Gandusari,2.153268,0.569862,2.026938,4.750067
4,Garum,1.807207,0.569862,2.026938,4.404006
5,Kademangan,1.663015,0.569862,0.405388,2.638264
6,Kanigoro,1.360211,0.569862,2.026938,3.957011
7,Kesamben,1.547661,0.569862,2.026938,4.144461
8,Nglegok,2.042720,0.569862,2.026938,4.639520
9,Panggungrejo,1.970624,0.439608,2.026938,4.437170


### Total topografi V5 * Bobot AHP topografi

In [18]:
# Mengalikan kolom 'total_skor' dengan bobot 'iklim'
total_skor_topo_V5 = skor_topo_V5.copy()
total_skor_topo_V5['total_skor_topo_V5'] = total_skor_topo_V5['total_skor'] * bobot_utama_dict['topografi']
total_skor_topo_V5


,nama_lahan,ketinggian,kemiringan,drainase,total_skor,total_skor_topo_V5
0,Bakung,1.249664,0.569862,2.026938,3.846464,0.438391
1,Binangun,1.648595,0.569862,2.026938,4.245395,0.483858
2,Doko,2.158074,0.569862,2.026938,4.754874,0.541924
3,Gandusari,2.153268,0.569862,2.026938,4.750067,0.541376
4,Garum,1.807207,0.569862,2.026938,4.404006,0.501935
5,Kademangan,1.663015,0.569862,0.405388,2.638264,0.300689
6,Kanigoro,1.360211,0.569862,2.026938,3.957011,0.450990
7,Kesamben,1.547661,0.569862,2.026938,4.144461,0.472354
8,Nglegok,2.042720,0.569862,2.026938,4.639520,0.528777
9,Panggungrejo,1.970624,0.439608,2.026938,4.437170,0.505715
